# YOLACT Google Colab Notebook
Inference on images and video with [YOLACT](https://github.com/dbolya/yolact).

## Google Colab
This notebook is set up to work inside Google Colab, which is a free, Linux-based Jupyter Notebook environment hosted in the cloud.

If you try it outside of Google Colab I'm not sure if it will work. I originally had problems running locally on my Windows machine, which is why I switched to this.

### Runtime Setup
- Go to Runtime > Change Runtime Type
- Choose GPU (TPU won't work)

## YOLACT
YOLACT is a state of the art, real-time, single shot object segmentation algorithm detailed in these papers:  
- [YOLACT: Real-time Instance Segmentation](https://arxiv.org/abs/1904.02689)
- [YOLACT++: Better Real-time Instance Segmentation](https://arxiv.org/abs/1912.06218)

**Big thanks to the authors: Daniel Bolya, Chong Zhou, Fanyi Xiao, Yong Jae Lee!**

## Immersive Limit
This notebook was created by Adam Kelly, instructor/creator of [immersivelimit.com](https://www.immersivelimit.com).
- Subscribe for lots more awesome AI and 3D content on the [YouTube channel](https://www.youtube.com/c/immersivelimit)
- Check out https://www.immersivelimit.com/connect for more ways to follow/connect 😀

# Initial Setup
First, we install some Python libraries. Google Colab probably has these requirements already, but better to include them than to break later.

In [ ]:
# Cython needs to be installed before pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib
!pip install sk-video>=1.1.8


In [ ]:
# Downgrade torch to accommodate DCNv2
!pip install torchvision==0.5.0
!pip install torch==1.4.0


     |████████████████████████████████| 4.0 MB 12.2 MB/s 
     |████████████████████████████████| 753.4 MB 7.2 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.


## Clone YOLACT from GitHub
Let's get that github repo! If anything in this notebook breaks, check the repo in case they've made updates.

In [ ]:
# Make sure we're in the top folder
%cd /home/bubble/work_dir/

# Clone the repo
!git clone https://github.com/dbolya/yolact.git

[Errno 2] No such file or directory: '/home/bubble/work_dir/'
/content
Cloning into 'yolact'...
remote: Enumerating objects: 2936, done.
remote: Total 2936 (delta 0), reused 0 (delta 0), pack-reused 2936
Receiving objects: 100% (2936/2936), 21.20 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (2003/2003), done.


## DCNv2
The DCNv2 external library is needed for this to work, so we'll build it now.

**IF YOU GET CUDA ERRORS** You need to change the Runtime of this notebook to "GPU" as mentioned at the top of the notebook.

In [ ]:
# Change to the right directory
%cd yolact/external/DCNv2

# Build DCNv2
!python setup.py build develop

/content/yolact/external/DCNv2
running build
running build_ext
building '_ext' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/yolact
creating build/temp.linux-x86_64-3.7/content/yolact/external
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src/cuda
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/content/yolact/exte

## Pretrained Weights
In order to run inference, we'll need some pre-trained weights. The creator of the GitHub repo shared them on Google Drive. We're going to use a [helpful tool](https://github.com/chentinghao/download_google_drive) made by [chentinghao](https://github.com/chentinghao) to easily access the Drive file from Colab.

If this stops working, it's probably because the weights moved to a different location. Check the YOLACT github repo to see where they went.

In [ ]:
# Make sure we're in the top folder
%cd /home/bubble/work_dir

# Clone the repo
!git clone https://github.com/chentinghao/download_google_drive.git

# Create a new directory for the pre-trained weights
!mkdir -p /home/bubble/work_dir/yolact/weights

# Download the file
!python ./download_google_drive/download_gdrive.py 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP ./yolact/weights/yolact_plus_resnet50_54_800000.pth

[Errno 2] No such file or directory: '/home/bubble/work_dir'
/content/yolact/external/DCNv2
Cloning into 'download_google_drive'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
Traceback (most recent call last):
  File "./download_google_drive/download_gdrive.py", line 57, in <module>
    download_file_from_google_drive(file_id, destination)
  File "./download_google_drive/download_gdrive.py", line 45, in download_file_from_google_drive
    save_response_content(response, destination)    
  File "./download_google_drive/download_gdrive.py", line 27, in save_response_content
    with open(destination, "wb") as f:
FileNotFoundError: [Errno 2] No such file or directory: './yolact/weights/yolact_plus_resnet50_54_800000.pth'


# Get Test Images
We'll download a few test images from the [COCO dataset](http://cocodataset.org/#explore). Feel free to try out your own images as well, but know that there are only 80 categories and they're not intended to cover everything, so if you upload a picture of a snake, a mouse, and a cat, only the cat will be detected.

In [ ]:
# Make a new directory for the test images
!mkdir /home/bubble/work_dir/test_images

# Move to the new directory
%cd /home/bubble/work_dir/test_images

# Make a list of URLs from the COCO dataset
get_imgs = ['http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg',
            'http://farm3.staticflickr.com/2835/9520716824_51220b13d0_z.jpg',
            'http://farm3.staticflickr.com/2817/10105052924_0c8d79e62c_z.jpg',
            'https://farm3.staticflickr.com/2637/3679968895_37a79e55b3_z.jpg',
            'https://farm8.staticflickr.com/7137/7705349298_6bea29cf1c_z.jpg',
            'http://farm5.staticflickr.com/4105/4981945644_3f162314ed_z.jpg',
            'http://farm1.staticflickr.com/40/101757885_a8fc40e273_z.jpg',
            'http://farm3.staticflickr.com/2654/3897535041_fa8dd5570b_z.jpg']

# Download all the images
for img_url in get_imgs:
  !wget {img_url} -nc

# List the contents of the folder. Should have images.
!pwd
!ls

/home/bubble/work_dir/test_images
--2021-09-28 15:14:19--  http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg
Resolving farm8.staticflickr.com (farm8.staticflickr.com)... 99.84.14.145, 2600:9000:218e:3800:0:5a51:64c9:c681, 2600:9000:218e:e200:0:5a51:64c9:c681, ...
Connecting to farm8.staticflickr.com (farm8.staticflickr.com)|99.84.14.145|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg [following]
--2021-09-28 15:14:19--  https://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg
Connecting to farm8.staticflickr.com (farm8.staticflickr.com)|99.84.14.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘6946559805_7edf757660_z.jpg’

6946559805_7edf7576     [ <=>                ]  52.93K  --.-KB/s    in 0.007s  

2021-09-28 15:14:19 (7.36 MB/s) - ‘6946559805_7edf757660_z.jpg’ saved [54199]

--2021-09-28 15:1

# Run Inference on Images
Let's run inference. We do that with eval.py

In [ ]:
# Move up to the top level directory
%cd /home/bubble/work_dir

# Delete any previous output_images folder (in case we're running this more than once)
!rm /home/bubble/work_dir/output_images -r

# Create a new directory for output images
!mkdir -p /home/bubble/work_dir/output_images

# Run inference using our pre-trained weights on all images in the directory
!python ./yolact/eval.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth --config=yolact_plus_resnet50_config --score_threshold=0.15 --top_k=15 --images=test_images:output_images

/home/bubble/work_dir
rm: cannot remove '/home/bubble/work_dir/output_images': No such file or directory
python3: can't open file './yolact/eval.py': [Errno 2] No such file or directory


## Display Output Images
The last step saved output images, so we'll display those now.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

output_images = Path('output_images')

def show_image(img_path):
  img = cv2.imread(img_path)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.show()

# Iterate through all of the output images and display them
for img_path in output_images.iterdir():
  print(img_path)
  show_image(str(img_path))

# Run Inference on Video
I can't figure out how to get Google Colab to show the output video in the notebook, so I've uploaded the video to YouTube. The mask quality is incredible. There are some false positives because it thinks the rocks are sheep, but we'll give it a pass because it's never been told what a rock is before (COCO dataset doesn't contain rocks) and the rocks do look a bit sheepish.


In [ ]:
from IPython.display import HTML
HTML('<iframe width="1280" height="720" src="https://www.youtube.com/embed/bQgtmbzN7jg" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

If you'd like to process your own video, you can modify the steps below.

In [ ]:
# Make a new directory for the test images
!mkdir -p /home/bubble/work_dir/test_video

# Move to the top level directory
%cd /home/bubble/work_dir

# Download the file
# https://drive.google.com/file/d/1I4fivRLniVNt_LcWqhQNehTSdEkZ4ytW/view?usp=sharing
# Note that file_id is extracted from the share URL above, it'll be different for every file
file_id = "12iSVHTs9GnWECjOZ7aqDbfHAsBkE2Zx4"
file_path = "/home/bubble/work_dir/test_video/road01.mp4" # <- this is the destination path

#!python /home/bubble/work_dir/download_google_drive/download_gdrive.py {file_id} {file_path}

!ls /home/bubble/work_dir/test_video

/home/bubble/work_dir
road01.mp4


Now we run inference.

In [ ]:
%cd /home/bubble/work_dir
# !mkdir -p /home/bubble/work_dir/output_videos
output_path = "/home/bubble/work_dir/output_videos/road01after.mp4"
!python ./yolact/eval1.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth  --config=yolact_plus_resnet50_config --output_coco_json --bbox_det_file="/content/" --score_threshold=0.15 --top_k=15 --video_multiframe=11 --traffic_video={file_path}:{output_path} --detect 

/home/bubble/work_dir
Loading model... Done.
./yolact/eval1.py:914: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  points = np.array(objects)[:, 0:3]
0
./yolact/eval1.py:1063: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  path = np.array(path)[:, 1].tolist()
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
10

In [ ]:
%cd /home/bubble/work_dir
# !mkdir -p /content/output_videos
# output_path = "/content/output_videos/road01after.mp4"
!python ./yolact/eval.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth  --config=yolact_plus_resnet50_config  --score_threshold=0.15 --top_k=15  --video=0 --detect

/home/bubble/work_dir
Loading model... Done.
Initializing model... Done.

Press Escape to close.
Processing FPS: 16.35 | Video Playback FPS: 0.00 | Frames in Buffer: 23    QObject::moveToThread: Current thread (0x7f5554002160) is not the object's thread (0x7f55540c4590).
Cannot move to target thread (0x7f5554002160)

Processing FPS: 16.25 | Video Playback FPS: 0.00 | Frames in Buffer: 25    QObject::moveToThread: Current thread (0x7f5554002160) is not the object's thread (0x7f55540c4590).
Cannot move to target thread (0x7f5554002160)

QObject::moveToThread: Current thread (0x7f5554002160) is not the object's thread (0x7f55540c4590).
Cannot move to target thread (0x7f5554002160)

QObject::moveToThread: Current thread (0x7f5554002160) is not the object's thread (0x7f55540c4590).
Cannot move to target thread (0x7f5554002160)

QObject::moveToThread: Current thread (0x7f5554002160) is not the object's thread (0x7f55540c4590).
Cannot move to target thread (0x7f5554002160)

Processing FPS: 16

## Download the output file
In this part, we'll mount your Google Drive and you can copy the file somewhere over there. I tried using the google.colab.files library to download the file, but it doesn't seem to like big video files.

- You will need to authorize your Drive to connect, then it basically just links your drive as if it's a folder on this cloud computer.
- **Be careful** because if you start deleting/modifying files in this folder, you will modify your personal Google Drive.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
!mkdir -p /content/drive/My\ Drive/YOLACT_output
%cd /content/drive/My\ Drive/YOLACT_output
!ls

In [ ]:
!cp {output_path} /content/drive/My\ Drive/YOLACT_output/giraffes_full_720_output.mp4

## Find the Video on Google Drive
Now you should be able to view the file on your Google Drive in the YOLACT_output folder.

# Conclusion
🎉 Woohoo! You did it! 🎉

Thanks for checking out the notebook, I hope it was helpful!

**If you liked it, consider sharing it!** YOLACT is pretty awesome, so more people should know about it.


### Want to learn how to Train on a custom dataset?

Check out this tutorial:

[Train YOLACT with a Custom COCO Dataset | Immersive Limit](https://www.immersivelimit.com/tutorials/train-yolact-with-a-custom-coco-dataset)